<a href="https://colab.research.google.com/github/anandchauhan21/pomegranate_disease_prediction/blob/main/LeafdetectCreateModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load libraries
import cv2
import numpy as np
from keras.layers import Dense
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from pylab import *
from skimage.color import rgb2hsv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import patches
import os
from google.colab.patches import cv2_imshow

print('Libraries imported...')

Libraries imported...


In [ ]:
## leaf detect
def LeafDetect(I):
    Ig = cv2.GaussianBlur(I, (19, 19), 0)  # apply gaussian blur
    Ihsv = rgb2hsv(Ig)  # convert to HSV
    Ih = np.uint8(255 * Ihsv[:, :, 0])  # convert range 0-1 to 0-255
    # Is = np.uint8(255 * Ihsv[:, :, 1])

    # plt.imshow(Ih, cmap='gray')
    # plt.title('hue image')
    # plt.show()

    Ib = 255 * np.uint8((Ih < 200) & (Ih > 50))  # apply hue saturation range filtering
    # plt.imshow(Ib, cmap='gray')
    # plt.title('mask1')
    # plt.show()

    kernel = np.ones((20, 20), np.uint8)  # create erosion kernel
    Ie = cv2.erode(Ib, kernel, iterations=1)  # apply erosion
    # plt.imshow(Ie, cmap='gray')
    # plt.title('erode1')
    # plt.show()

    ## perform image closing using contour method
    # Ic = cv2.morphologyEx(Ie, cv2.MORPH_CLOSE, kernel)# apply morphological processing
    # plt.imshow(Ic, cmap='gray')
    # plt.title('close')
    # plt.show()

    # Filter using contour area and remove small noise
    cnts = cv2.findContours(Ib, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < 10000:
            cv2.drawContours(Ib, [c], -1, (0, 0, 0), -1)

    # Morph close and invert image
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))  # create elliptical kernel
    Iclose = np.uint8(cv2.morphologyEx(Ib, cv2.MORPH_CLOSE, kernel, iterations=3))  # apply morphology process
    # print(Iclose[50,:])
    # plt.imshow(Iclose, cmap='gray')
    # plt.title('countour')
    # plt.show()

    loc = np.array(np.where(Iclose == 255))  # find the while pixels
    locr = loc[0, :]  # read raw locations
    locc = loc[1, :]  # read column location

    xmin = np.min(locc)
    xmax = np.max(locc)
    ymin = np.min(locr)
    ymax = np.max(locr)
    lwidth = xmax - xmin
    lhght = ymax - ymin

    #    fig, ax = plt.subplots(1, figsize=(16, 16))
    #    ax.axis('off')
    #    ax.imshow(I2, cmap="gray")
    #    rect = patches.Rectangle((xmin, ymin), lwidth, lhght, fill=False, linewidth=2, alpha=0.7, edgecolor='r',facecolor='none')
    #    ax.add_patch(rect)
    # plt.waitforbuttonpress()

    Igray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)  # convert to grayscale
    Icut = Igray[ymin:ymax, xmin:xmax]  # crop the image
    return Icut


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
##------------------------  folder processing  s--------------------
current_path = os.getcwd()
Colab_Path = os.path.join(current_path,'drive','MyDrive')
ImageDB_Path = os.path.join(Colab_Path,'leaf')
print(ImageDB_Path)


/content/drive/MyDrive/leaf


In [ ]:
############ IMPORTING DATA/IMAGES FROM FOLDERS
RH = 250 # resize image height
RW = 250 # resize image width
count = 0
images = []  # LIST CONTAINING ALL THE IMAGES
classNo = []  # LIST CONTAINING ALL THE CORRESPONDING CLASS ID OF IMAGES
myList = os.listdir(ImageDB_Path)  # print the path
print('*********   Pomo Leaf based desease detection ************')
print("Total Classes Detected:", len(myList))  # list the total class (types of pomo, here only 2)
noOfClasses = len(myList)  # length of list
print("Importing Classes .......")

for FolderName in range(0, noOfClasses):  # read all folders
    PicLocations = os.path.join(ImageDB_Path,str(FolderName))
    myPicList = os.listdir(PicLocations)  # read folder
    print('\n\n *********************Reading folder: ', FolderName, end=", ******************************\n")
    for Files in myPicList:  # read all images in a folder
        tpath = os.path.join(PicLocations,Files)
        #tpath = ImageDB_Path + "\\" + str(FolderName) + "\\" + Files # create path of image
        print(tpath)
        curImg = cv2.imread(tpath)  # read image
        Igray = cv2.cvtColor(curImg, cv2.COLOR_BGR2GRAY)  # convert rgb 2 gray
        Icut = LeafDetect(curImg)  # detect pomo area in that image
        # Icut2 = Igray
        Icut2 = Icut / 255  # scale the image
        Ir = cv2.resize(Icut2, (RH, RW))  # resize the image to 125x125

        images.append(Ir)  # append to image database
        classNo.append(FolderName)  # append classes to classes database

print(" ")
print("Total Images in Images List = ", len(images))
print("Total IDS in classNo List= ", len(classNo))

*********   Pomo Leaf based desease detection ************
Total Classes Detected: 2
Importing Classes .......


 *********************Reading folder:  0, ******************************
/content/drive/MyDrive/leaf/0/DSC02320.JPG
/content/drive/MyDrive/leaf/0/DSC02354.JPG
/content/drive/MyDrive/leaf/0/DSC02341.JPG
/content/drive/MyDrive/leaf/0/DSC02356.JPG
/content/drive/MyDrive/leaf/0/DSC02343.JPG
/content/drive/MyDrive/leaf/0/DSC02316.JPG
/content/drive/MyDrive/leaf/0/DSC02334.JPG
/content/drive/MyDrive/leaf/0/DSC02402.JPG
/content/drive/MyDrive/leaf/0/DSC02338.JPG
/content/drive/MyDrive/leaf/0/DSC02357.JPG
/content/drive/MyDrive/leaf/0/DSC02403.JPG
/content/drive/MyDrive/leaf/0/DSC02406.JPG
/content/drive/MyDrive/leaf/0/DSC02358.JPG
/content/drive/MyDrive/leaf/0/DSC02319.JPG
/content/drive/MyDrive/leaf/0/DSC02321.JPG
/content/drive/MyDrive/leaf/0/DSC02355.JPG
/content/drive/MyDrive/leaf/0/DSC02352.JPG
/content/drive/MyDrive/leaf/0/DSC02310.jpg
/content/drive/MyDrive/leaf/0/DSC02364.J

In [ ]:
#### CONVERT TO NUMPY ARRAY
images = np.array(images)
classNo = np.array(classNo)
print('image size=' + str(images.shape))

#### SPLITTING THE DATA
testRatio = 0.2 # test image ratio
valRatio = 0.2 # validation image ratio
imageDimensions = (RH, RW, 3)  # image dimension for processing
batchSizeVal = 2 # batch size of processing
epochsVal = 30  # no of epoch for making algorithm best fit
stepsPerEpochVal = 2000
print("Parameters initialised")


image size=(109, 250, 250)
Parameters initialised


In [ ]:
# split the training and test images
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=valRatio)
print('Training Image size=' + str(X_train.shape))
print('Test Image size=' + str(X_test.shape))
print('Validation images size=' + str(X_validation.shape))

X_train = np.array(list(X_train)) # convert to numpy array
X_test = np.array(list(X_test)) # convert to numpy array
X_validation = np.array(list(X_validation)) # convert to numpy array

#### RESHAPE IMAGE DATABASES
print(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
print('Image data reshaping done')

#### ONE HOT ENCODING OF MATRICES
y_train = to_categorical(y_train, noOfClasses)
y_test = to_categorical(y_test, noOfClasses)
y_validation = to_categorical(y_validation, noOfClasses)


Training Image size=(69, 250, 250)
Test Image size=(22, 250, 250)
Validation images size=(18, 250, 250)
69 250 250
Image data reshaping done


In [ ]:
#### CREATING THE MODEL
def myModel():
    noOfFilters = 60  # no of filters to be used
    sizeOfFilter1 = (5, 5)  # first convolution filter size
    sizeOfFilter2 = (3, 3)  # second convolution filter size
    sizeOfPool = (2, 2)
    noOfNodes = 500  # no of NN nodes

    model = Sequential()  # initialise model type
    model.add((Conv2D(noOfFilters, sizeOfFilter1, input_shape=(imageDimensions[0],
                                                               imageDimensions[1], 1),
                      activation='relu')))  # add layer 1
    model.add((Conv2D(noOfFilters, sizeOfFilter1, activation='relu')))  # layer 2
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add((Conv2D(noOfFilters // 2, sizeOfFilter2, activation='relu')))  # layer 3
    model.add((Conv2D(noOfFilters // 2, sizeOfFilter2, activation='relu')))  # layer 4
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add(Dropout(0.5))

    model.add(Flatten())  # convert 2D feature matrix to 1D vector
    model.add(Dense(noOfNodes, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses, activation='softmax'))

    model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])  # compile the model
    return model


In [ ]:
model = myModel()  # create the model
print('++++++++++++++ MODEL CREATED ++++++++++')
print(model.summary())


++++++++++++++ MODEL CREATED ++++++++++
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 246, 246, 60)      1560      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 242, 242, 60)      90060     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 121, 121, 60)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 119, 119, 30)      16230     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 117, 117, 30)      8130      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 58, 58, 30)        0         
_________________________________________________________________
dropout (Dropout

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# save model
crnt_path ='/content/drive/'
FLoc= os.path.join(crnt_path,'MyDrive','Colab Notebooks','model-leaf.h5')
print('Model file location', FLoc)
model.save(FLoc)
print('Creted model saved as, model-leaf.h5')

Model file location /content/drive/MyDrive/Colab Notebooks/model-leaf.h5
Creted model saved as, model-leaf.h5
